In [175]:
#!conda install -c anaconda beautifulsoup4 
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


### 1.  store the url link into url varaible, then use pandas library to store raw data from web.

In [176]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

table = pd.read_html(url)
table = table[0]

In [177]:
table.head() # take a quick look of this df

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [178]:
table.columns = ['Postcode','Borough','Neighbourhood'] # rename col name


table = table.drop(table.index[0]) #remove index 0


table = table.reset_index(drop= True)

In [179]:
table.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [180]:
df = table.replace('Not assigned', np.nan) # replace not assigned with NaN, easier to handle df
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [181]:
df = df[pd.notnull(df['Borough'])] # filter out NaN values in Borough while kept nan values in Neighbourhood 
df.head()
#df = df.dropna(how= 'any', axis = 0)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [182]:
df.reset_index(drop = True, inplace = True)
df.head()


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [183]:
df.replace(np.nan,"Queen's Park",inplace = True) # since we only have one nan value, let Borough = Neighbourhood
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### 2. use comma seprate string type value

In [238]:
df1 = df.groupby(['Postcode','Borough'])['Neighbourhood'].agg(','.join) 
df1.sort_values(ascending = True,inplace = True)

In [239]:
df1[0:10]

Postcode  Borough         
M5H       Downtown Toronto                               Adelaide,King,Richmond
M1S       Scarborough                                                 Agincourt
M1V       Scarborough         Agincourt North,L'Amoreaux East,Milliken,Steel...
M9V       Etobicoke           Albion Gardens,Beaumond Heights,Humbergate,Jam...
M8W       Etobicoke                                       Alderwood,Long Branch
M3H       North York              Bathurst Manor,Downsview North,Wilson Heights
M2K       North York                                            Bayview Village
M5M       North York                           Bedford Park,Lawrence Manor East
M5E       Downtown Toronto                                          Berczy Park
M1N       Scarborough                                Birch Cliff,Cliffside West
Name: Neighbourhood, dtype: object

In [240]:
lai = pd.DataFrame({'Neighbourhood':df1})  #store groupby object into dataframe
lai.head()

,,Neighbourhood
Postcode,Borough,
M5H,Downtown Toronto,"Adelaide,King,Richmond"
M1S,Scarborough,Agincourt
M1V,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel..."
M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
M8W,Etobicoke,"Alderwood,Long Branch"


### 3. Convert Multi_index into cols

In [241]:
# however, we have 2 cols of indexs, the following code will loop out the multi-index then add to columns.
p = []
b = []
for i ,j in lai.index: 
    p.append(i)
    b.append(j)
    
    
lai['Postcode'] = p    
lai['Borough'] = b    
lai.reset_index(drop = True,inplace = True)
lai.head() #result

,Neighbourhood,Postcode,Borough
0,"Adelaide,King,Richmond",M5H,Downtown Toronto
1,Agincourt,M1S,Scarborough
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",M1V,Scarborough
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",M9V,Etobicoke
4,"Alderwood,Long Branch",M8W,Etobicoke
